In [76]:
import sqlite3
import pandas as pd

In [77]:
database = r"zack_database.db"
conn = sqlite3.connect(database)

t_zacks_fc = pd.read_sql_query("SELECT ticker, zacks_sector_code, per_end_date, per_type, eps_diluted_net_basic, lterm_debt_net_tot FROM t_zacks_fc", conn)
t_zacks_fr = pd.read_sql_query("SELECT ticker, per_end_date, per_type, ret_invst, tot_debt_tot_equity FROM t_zacks_fr", conn)
t_zacks_mktv = pd.read_sql_query("SELECT ticker, per_end_date, per_type, mkt_val FROM t_zacks_mktv", conn)
t_zacks_shrs = pd.read_sql_query("SELECT ticker, per_end_date, per_type, shares_out FROM t_zacks_shrs", conn)

conn.close()

## Inner

In [78]:
merged_df = t_zacks_fc.merge(t_zacks_fr, on=['ticker', 'per_end_date', 'per_type'], how='inner')
merged_df = merged_df.merge(t_zacks_mktv, on=['ticker', 'per_end_date', 'per_type'], how='inner')
merged_df = merged_df.merge(t_zacks_shrs, on=['ticker', 'per_end_date', 'per_type'], how='inner')
merged_df['zacks_sector_code'] = merged_df.groupby('ticker')['zacks_sector_code'].transform(lambda x: x.ffill().bfill())
merged_df

,ticker,zacks_sector_code,per_end_date,per_type,eps_diluted_net_basic,lterm_debt_net_tot,ret_invst,tot_debt_tot_equity,mkt_val,shares_out
0,EPC,1.0,2024-03-31,Q,0.72,-11.000,1.2405,0.8860,1929.42,49.93
1,EPAM,10.0,2024-03-31,Q,1.97,-0.589,3.3198,0.0074,16015.99,57.99
2,KD,16.0,2024-03-31,Q,-0.19,-150.000,-1.0628,2.8859,5007.05,230.10
3,PARR,12.0,2024-03-31,Q,-0.06,-18.565,-0.1914,0.4979,2199.16,59.34
4,PARA,2.0,2024-03-31,Q,-0.87,14607.000,-1.4969,0.6540,7844.45,666.48
...,...,...,...,...,...,...,...,...,...,...
6296,MDXG,4.0,2024-09-30,Q,0.05,-31.467,3.9583,0.1051,868.45,146.95
6297,GUTS,4.0,2024-09-30,Q,-0.48,-0.081,-28.7367,0.6025,121.26,47.93
6298,BATL,12.0,2024-09-30,Q,0.34,-52.383,16.9429,4.0811,109.76,16.46
6299,EVCM,10.0,2024-09-30,Q,-0.05,-4.125,-0.7074,0.6864,1908.86,184.25


In [86]:
# 1. Earnings per Share (EPS)
merged_df['EPS'] = merged_df['eps_diluted_net_basic']

# 2. Return on Investments (ROI)
merged_df['ROI'] = merged_df['ret_invst']

# 3. Total Debt to Equity Ratio
merged_df['Debt_to_Equity_Ratio'] = merged_df['tot_debt_tot_equity']

# 4. Net Long-Term Debt
merged_df['Net_Long_Term_Debt'] = merged_df['lterm_debt_net_tot']

# 5. Market Capitalization
merged_df['Market_Capitalization'] = merged_df['mkt_val']

# 6. Price-to-Earnings Ratio (P/E)
merged_df['P_E_Ratio'] = merged_df['mkt_val'] / (merged_df['eps_diluted_net_basic'] * merged_df['shares_out'])

# 7. Market Cap to Shares Outstanding
merged_df['Market_Cap_per_Share'] = merged_df['mkt_val'] / merged_df['shares_out']

# 8. Earnings Growth Rate
merged_df = merged_df.sort_values(by=['ticker', 'per_end_date'])
merged_df['Previous_EPS'] = merged_df.groupby('ticker')['eps_diluted_net_basic'].shift(1)
merged_df['Earnings_Growth_Rate'] = (merged_df['eps_diluted_net_basic'] - merged_df['Previous_EPS']) / merged_df['Previous_EPS']

# 9. Debt to Market Cap Ratio
merged_df['Debt_to_Market_Cap_Ratio'] = merged_df['lterm_debt_net_tot'] / merged_df['mkt_val']

# 10. Sector Relative Valuation (P/E to Sector Average)
sector_pe_avg = merged_df.groupby('zacks_sector_code')['P_E_Ratio'].transform('mean')
merged_df['Sector_Relative_Valuation'] = merged_df['P_E_Ratio'] / sector_pe_avg

# 11. Relative Debt to Equity (compared with sector averages)
sector_de_avg = merged_df.groupby('zacks_sector_code')['tot_debt_tot_equity'].transform('mean')
merged_df['Relative_Debt_to_Equity'] = merged_df['tot_debt_tot_equity'] / sector_de_avg

# Display the updated DataFrame with new columns
risk_metrics_inner = merged_df[['ticker', 'zacks_sector_code', 'per_end_date', 'EPS', 'ROI', 'Debt_to_Equity_Ratio', 'Net_Long_Term_Debt', 
                           'Market_Capitalization', 'P_E_Ratio', 'Market_Cap_per_Share', 
                           'Earnings_Growth_Rate', 'Debt_to_Market_Cap_Ratio', 
                           'Sector_Relative_Valuation', 'Relative_Debt_to_Equity']]
risk_metrics_inner

,ticker,zacks_sector_code,per_end_date,EPS,ROI,Debt_to_Equity_Ratio,Net_Long_Term_Debt,Market_Capitalization,P_E_Ratio,Market_Cap_per_Share,Earnings_Growth_Rate,Debt_to_Market_Cap_Ratio,Sector_Relative_Valuation,Relative_Debt_to_Equity
1842,AAGR,1.0,2024-03-31,-0.22,52.4350,-0.2046,-0.045,3.71,-1.786402,0.393008,NaN,-0.012129,-0.0,-0.091348
1841,AAL,15.0,2024-03-31,-0.48,-1.3728,-5.8782,-625.000,10070.99,-31.979194,15.350013,NaN,-0.062059,-0.0,-12.948845
3617,AAL,15.0,2024-06-30,1.01,3.1324,-6.6911,-1309.000,7440.46,11.217896,11.330075,-3.104167,-0.175930,0.0,-14.739549
5316,AAL,15.0,2024-09-30,-0.23,-0.6958,-6.5208,-1446.000,7386.15,-48.869623,11.240013,-1.227723,-0.195772,-0.0,-14.364402
1843,AAOI,10.0,2024-03-31,-0.60,-8.5166,0.5189,-0.465,536.79,-23.099664,13.859799,NaN,-0.000866,-0.0,-5.516075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1844,ZVRA,4.0,2024-03-31,-0.40,-18.1036,0.8808,42.998,251.87,-14.498618,5.799447,NaN,0.170715,-0.0,-0.663418
3616,ZVRA,4.0,2024-06-30,-0.48,-21.9424,1.7959,16.941,205.08,-10.209080,4.900358,0.200000,0.082607,-0.0,-1.352670
1845,ZWS,7.0,2024-03-31,0.19,1.6126,0.3070,-0.200,5780.16,176.154573,33.469369,NaN,-0.000035,0.0,0.361322
3615,ZWS,7.0,2024-06-30,0.27,2.1685,0.3108,-0.400,5025.30,108.887979,29.399754,0.421053,-0.000080,0.0,0.365794


In [87]:
risk_metrics_inner['ticker'].unique()

array(['AAGR', 'AAL', 'AAOI', ..., 'ZTS', 'ZVRA', 'ZWS'], dtype=object)

In [88]:
len(risk_metrics_inner['ticker'].unique())

2621

In [90]:
risk_metrics_inner[risk_metrics_inner['ticker'] == 'AAOI']

,ticker,zacks_sector_code,per_end_date,EPS,ROI,Debt_to_Equity_Ratio,Net_Long_Term_Debt,Market_Capitalization,P_E_Ratio,Market_Cap_per_Share,Earnings_Growth_Rate,Debt_to_Market_Cap_Ratio,Sector_Relative_Valuation,Relative_Debt_to_Equity
1843,AAOI,10.0,2024-03-31,-0.60,-8.5166,0.5189,-0.465,536.79,-23.099664,13.859799,NaN,-0.000866,-0.0,-5.516075
3619,AAOI,10.0,2024-06-30,-0.66,-9.9227,0.5319,-0.465,322.55,-12.560065,8.289643,0.100000,-0.001442,-0.0,-5.654269
5317,AAOI,10.0,2024-09-30,-0.42,-6.1413,0.5023,-0.465,585.22,-34.067994,14.308557,-0.363636,-0.000795,-0.0,-5.339611
